# Creating Normalized Tables

## Imports

In [1]:
import psycopg2

## Make Connection to DB

In [2]:
try:
    conn = psycopg2.connect("dbname=####")
except psycopg2.Error as e:
    print('Error: Unable to establish a connection')
    print(e)
    
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print('Error: Cursor not connected')
    print(e)

conn.set_session(autocommit = True)

## Details of Table

* **Table Name**: music_library
* **Column 0**: Album ID
* **Column 1**: Album Name
* **Column 2**: Artist Name
* **Column 3**: Year
* **Column 4**: List of Songs

## Create Table (Not Normalized)

In [3]:
query = 'CREATE TABLE IF NOT EXISTS music_library '
query += '(album_id int, album_name varchar, artist_name varchar, year int, songs text[]);'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print('Error: Table not created')
    print(e)

In [4]:
query = 'INSERT into music_library (album_id, album_name, artist_name, year, songs) '
query += 'VALUES (%s, %s, %s, %s, %s)'    

try:
    cur.execute(query, (1, 
                        'Rubber Soul', 
                        'The Beatles', 
                        1965, 
                        ['Michelle', 'Think for Yourself', 'In my Life']))
except psycopg2.Error as e:
    print('Error: Row not inserted')
    print(e)  

In [5]:
try:
    cur.execute(query, (2, 
                        'Let It Be', 
                        'The Beatles', 
                        1970,
                        ['Let It Be', 'Across the Universe']))
except psycopg2.Error as e:
    print('Error: Row not inserted')
    print(e)

### Validate Insertion

In [6]:
try:
    cur.execute('SELECT * FROM music_library;')
except psycopg2.Error as e:
    print('Error: SELECT *')
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965, ['Michelle', 'Think for Yourself', 'In my Life'])
(2, 'Let It Be', 'The Beatles', 1970, ['Let It Be', 'Across the Universe'])
(1, 'Rubber Soul', 'The Beatles', 1965, ['Michelle', 'Think for Yourself', 'In my Life'])
(2, 'Let It Be', 'The Beatles', 1970, ['Let It Be', 'Across the Universe'])


## Convert Table to `1NF`

In [7]:
query = 'CREATE TABLE IF NOT EXISTS music_library_1nf '
query += '(album_id int, album_name varchar, artist_name varchar, year int, song_name varchar);'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print('Error: Table not created')
    print(e)

In [8]:
def insert_row(table, album_id, album_name, artist_name, year, song_name):
    query = f'INSERT into {table} (album_id, album_name, artist_name, year, song_name) '
    query += 'VALUES (%s, %s, %s, %s, %s)'    

    try:
        cur.execute(query, (album_id, 
                            album_name, 
                            artist_name, 
                            year, 
                            song_name))
    except psycopg2.Error as e:
        print('Error: Row not inserted')
        print(e)  

In [9]:
insert_row('music_library_1nf',
           1,
           'Rubber Soul',
           'The Beatles',
           1965,
           'Michelle')

In [10]:
insert_row('music_library_1nf',
           1,
           'Rubber Soul',
           'The Beatles',
           1965,
           'Think for Yourself')

In [11]:
insert_row('music_library_1nf',
           1,
           'Rubber Soul',
           'The Beatles',
           1965,
           'In My Life')

In [12]:
insert_row('music_library_1nf',
           2,
           'Let It Be',
           'The Beatles',
           1970,
           'Let It Be')

In [13]:
insert_row('music_library_1nf',
           2,
           'Let It Be',
           'The Beatles',
           1970,
           'Across the Universe')

In [14]:
query = "DELETE FROM music_library_1nf WHERE album_id = 2 AND song_name = 'Think for Yourself'"

cur.execute(query)

In [15]:
try:
    cur.execute('SELECT DISTINCT * FROM music_library_1nf')
except psycopg2.Error as e:
    print(e)
    
row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965, 'Think for Yourself')
(1, 'Rubber Soul', 'The Beatles', 1965, 'Michelle')
(2, 'Let It Be', 'The Beatles', 1970, 'Let It Be')
(1, 'Rubber Soul', 'The Beatles', 1965, 'In My Life')
(2, 'Let It Be', 'The Beatles', 1970, 'Across the Universe')


## Convert to `2NF`
* Break data into two tables: `album_library` and `song_library`

### Create `album_library` table

In [16]:
query = 'CREATE TABLE IF NOT EXISTS album_library '
query += '(album_id int, album_name varchar, artist_name varchar, year int);'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [17]:
def insert_album(table, album_id, album_name, artist_name, year):
    query = f'INSERT into {table} (album_id, album_name, artist_name, year) '
    query += 'VALUES (%s, %s, %s, %s)'    

    try:
        cur.execute(query, (album_id, 
                            album_name, 
                            artist_name, 
                            year))
    except psycopg2.Error as e:
        print('Error: Row not inserted')
        print(e) 

In [18]:
insert_album('album_library',
             1,
             'Rubber Soul',
             'The Beatles',
             1965)

In [19]:
insert_album('album_library',
             2,
             'Let It Be',
             'The Beatles',
             1970)

In [20]:
query = 'SELECT * FROM album_library'
cur.execute(query)

row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965)
(2, 'Let It Be', 'The Beatles', 1970)
(1, 'Rubber Soul', 'The Beatles', 1965)
(2, 'Let It Be', 'The Beatles', 1970)


### Create `song_library` table

In [21]:
query = 'CREATE TABLE IF NOT EXISTS song_library '
query += '(song_id SERIAL, song_name varchar, album_id int)'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [22]:
def insert_song(table, song_name, album_id):
    query = f'INSERT into {table} (song_name, album_id) '
    query += 'VALUES (%s, %s)'    

    try:
        cur.execute(query, (song_name,
                            album_id))
    except psycopg2.Error as e:
        print('Error: Row not inserted')
        print(e) 
    

In [23]:
insert_song('song_library', 'Michelle', 1)

In [24]:
insert_song('song_library', 'Think for Yourself', 1)

In [25]:
insert_song('song_library', 'In My Life', 1)

In [26]:
insert_song('song_library', 'Let It Be', 2)

In [27]:
insert_song('song_library', 'Across the Universe', 2)

In [28]:
query = 'SELECT * FROM song_library'
try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)
    
row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

(1, 'Michelle', 1)
(2, 'Think for Yourself', 1)
(3, 'In My Life', 1)
(4, 'Let It Be', 2)
(5, 'Across the Universe', 2)
(6, 'Michelle', 1)
(7, 'Think for Yourself', 1)
(8, 'In My Life', 1)
(9, 'Let It Be', 2)
(10, 'Across the Universe', 2)
(11, 'Michelle', 1)
(12, 'Think for Yourself', 1)
(13, 'In My Life', 1)
(14, 'Let It Be', 2)
(15, 'Across the Universe', 2)


### Join Tables

In [29]:
query = 'SELECT * FROM album_library a JOIN song_library s ON s.album_id = a.album_id'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [30]:
row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965, 1, 'Michelle', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 2, 'Think for Yourself', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 3, 'In My Life', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 6, 'Michelle', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 7, 'Think for Yourself', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 8, 'In My Life', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 11, 'Michelle', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 12, 'Think for Yourself', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 13, 'In My Life', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 1, 'Michelle', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 2, 'Think for Yourself', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 3, 'In My Life', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 6, 'Michelle', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 7, 'Think for Yourself', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 8, 'In My Life', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 11, 'Michelle', 1)
(1, 'Rub

## Convert to `3NF`
* Organize the data into three tables:
    1. album_library2
    2. song_library
    3. artist_library

### Create Tables

In [31]:
query = 'CREATE TABLE IF NOT EXISTS album_library2 '
query += '(album_id SERIAL, album_name VARCHAR, artist_id INT, year INT)'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [32]:
query = 'CREATE TABLE IF NOT EXISTS song_library2 '
query += '(song_id SERIAL, song_name varchar, album_id int)'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [33]:
query = 'CREATE TABLE IF NOT EXISTS artist_library'
query += '(artist_id SERIAL, artist_name varchar)'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

### Insert Data Into Tables

#### Artists

In [34]:
query = "INSERT INTO artist_library (artist_name) VALUES ('The Beatles')"

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

#### Albums

In [35]:
def insert_album(table, album_name, artist_id, year):
    query = f"INSERT INTO {table} (album_name, artist_id, year) VALUES (%s, %s, %s)"
    
    try:
        cur.execute(query, (album_name, artist_id, year))
    except psycopg2.Error as e:
        print(e)

In [36]:
insert_album('album_library2', 'Rubber Soul', 1, 1965)

In [37]:
insert_album('album_library2', 'Let It Be', 1, 1970)

#### Songs

In [38]:
def insert_songs(table, album_id, song_name):
    query = f"INSERT INTO {table} (album_id, song_name) VALUES (%s, %s)"
    
    try:
        cur.execute(query, (album_id, song_name))
    except psycopg2.Error as e:
        print(e)

In [39]:
insert_songs('song_library2',1, 'Michelle')

In [40]:
insert_songs('song_library2', 1, 'Think for Yourself')

In [41]:
insert_songs('song_library2', 1, 'In My Life')

In [42]:
insert_songs('song_library2', 2, 'Let It Be')

In [43]:
insert_songs('song_library2', 2, 'Across the Universe')

### Validate Queries

In [44]:
def select(table):
    query = f'SELECT * FROM {table}'
    try:
        cur.execute(query)
    except psycopg2.Error as e:
        print(e)

    row = cur.fetchone()

    while row:
        print(row)
        row = cur.fetchone()

In [45]:
select('album_library2')

(1, 'Rubber Soul', 1, 1965)
(2, 'Let It Be', 1, 1970)
(3, 'Rubber Soul', 1, 1965)
(4, 'Let It Be', 1, 1970)


In [46]:
select('artist_library')

(1, 'The Beatles')
(2, 'The Beatles')


In [47]:
select('song_library2')

(1, 'Michelle', 1)
(2, 'Think for Yourself', 1)
(3, 'In My Life', 1)
(4, 'Let It Be', 2)
(5, 'Across the Universe', 2)


## Close Connection

In [1]:
cur.close()
conn.close()


NameError: name 'cur' is not defined